In [1]:
!pip install GPUtil

import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU not available.")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

# Change if needed to accomodate memory requirements!
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Set to the GPU ID (0 for T4)

!nvidia-smi # Verify notebook is running on a GPU

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=69073baaffd56eb46ea27f297a3a4ae8f68abfa23abbbdd7660fa681abdab1f6
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available!
Sat Feb  1 07:50:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=================

In [2]:
!pip install transformers bitsandbytes accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
from peft import PeftModel, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datetime import datetime

if 'COLAB_GPU' in os.environ:
    from google.colab import output
    output.enable_custom_widget_manager()

In [4]:
if 'COLAB_GPU' in os.environ:
    !huggingface-cli login
else:
    notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `fine-tuning-solutions-challenge` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active toke

In [5]:
  base_model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16,
  )

  '''
  NOTE:


  load_in_4bit: Enables loading the model using 4-bit quantization, reducing
  memory and computational costs.

  bnb_4bit_compute_dtype: Sets the computational data type for the 4-bit quantized
  model, controlling precision during inference or training.
  '''

  model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                              quantization_config=bnb_config,
                                              trust_remote_code=True,
                                              device_map="auto") # Helps with memory management

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

'''
the End-of-sentence (eos) token and the padding (pad) token.
'''

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# set the pad token to indicate that it's the end-of-sentence
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
# tokenized_train_dataset=[]
# for phrase in train_dataset:
#     tokenized_train_dataset.append(tokenizer(phrase['text']))

In [15]:
import random
# Define dataset paths
train_data_path = "./sample_data/training_data.jsonl" if 'COLAB_GPU' in os.environ else "training_data.jsonl"
val_data_path = "./sample_data/validation_data.jsonl" if 'COLAB_GPU' in os.environ else "validation_data.jsonl"

def sample_dataset(data_path, max_samples=5000):
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Random sampling
    if len(lines) > max_samples:
        lines = random.sample(lines, max_samples)
    return lines

# Function to process JSONL dataset, skipping bad
def process_jsonl(lines): # Changed: Accepts lines instead of file_path
    data = []
    for line_number, line in enumerate(lines, start=1): # Iterate through lines
        try:
            entry = json.loads(line)
            if "contents" in entry:
                # Extract user input (prompt) and model output (completion)
                user_text = entry["contents"][0]["parts"][0]["text"]
                model_text = entry["contents"][1]["parts"][0]["text"]
                data.append({
                    "prompt": user_text,
                    "completion": model_text
                })
        except json.JSONDecodeError as e:
            print(f"Skipping line {line_number} due to JSONDecodeError: {e}")
    return Dataset.from_list(data)

# Instead of directly using process_jsonl, first sample the data:
sampled_train_lines = sample_dataset(train_data_path)
sampled_val_lines = sample_dataset(val_data_path, max_samples=3000)  # 10% of training

# Load and process dataset
train_dataset = process_jsonl(sampled_train_lines)
val_dataset = process_jsonl(sampled_val_lines)

def tokenize_function(examples):
    # Process prompts and completions separately
    prompts = [str(p) for p in examples["prompt"]]
    completions = [str(c) for c in examples["completion"]]

    # Tokenize both with appropriate parameters
    model_inputs = tokenizer(
        prompts,
        padding="max_length",
        truncation=True,
        max_length=256,  # Adjust this based on your needs
        return_tensors=None  # Don't force tensor return in map
    )

    # Tokenize targets (completions)
    labels = tokenizer(
        completions,
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors=None
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization
train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=32,
    remove_columns=train_dataset.column_names  # Remove original columns
)
val_dataset = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [16]:
# gradient checkpointing to reduce memory usage for increased compute time
model.enable_input_require_grads()
model.gradient_checkpointing_enable()

# compressing the base model into a smaller, more efficient model
model = prepare_model_for_kbit_training(model)

In [17]:
config = LoraConfig(
    # rank of the update matrices
    # Lower rank results in smaller matrices with fewer trainable params
    r=16,

    # impacts low-rank approximation aggressiveness
    # increasing value speeds up training
    lora_alpha=32,

    # modules to apply the LoRA update matrices
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "down_proj",
        "up_proj",
        "o_proj"
    ],

    # determines LoRA bias type, influencing training dynamics
    bias="none",

    # regulates model regularization; increasing may lead to underfitting
    lora_dropout=0.1,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
# Print trainable parameters to verify setup
model.print_trainable_parameters()

trainable params: 18,464,768 || all params: 1,795,552,768 || trainable%: 1.0284


In [18]:
torch.cuda.empty_cache()  # Clear GPU memory

# Enable memory efficient attention if available in model
if hasattr(model, "enable_mem_efficient_attention"):
    model.enable_mem_efficient_attention()

### TWO IMPORTANT TRAINING PARAMETERS TO CONSIDER CHANGING

train_epochs_val = 1 # CHANGE VALUE AS NEEDED HERE!
'''
train_epochs_val is the times the model will iterate over the entire training
dataset. Increasing the value may allow the model to learn more from the data,
but be cautious of overfitting.
'''

learning_rate_val = 1e-4 # CHANGE VALUE AS NEEDED HERE!
'''
A higher learning_rate_val can lead to faster convergence, but it might
overshoot the optimal solution. Conversely, a lower value may result
in slower training but better fine-tuning.
'''


# Trainer setup
training_args = TrainingArguments(
        output_dir="./finetunedModel",       # directory where checkpoints are saved
        per_device_train_batch_size=4,       # number of samples processed in one forward/backward pass per GPU
        gradient_accumulation_steps=1,       # [default = 1] number of updates steps to accumulate the gradients for
        num_train_epochs=train_epochs_val,   # [IMPORTANT] number of times of complete pass through the entire training dataset
        learning_rate=learning_rate_val,     # [IMPORTANT] smaller LR for better finetuning
        bf16=False,                          # train parameters with this precision
        optim="paged_adamw_8bit",            # use paging to improve memory management of default adamw optimizer
        logging_dir="./logs",                # directory to save training log outputs
        save_strategy="steps",               # [default = "steps"] store after every iteration of a datapoint
        save_steps=200,                      # save checkpoint after number of iterations
        logging_steps = 10                   # specify frequency of printing training loss data
    )

# Create an instance of the Trainer class and assign it to trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset, # Add train_dataset
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# if use_cache is True, past key values are used to speed up decoding
# if applicable to model. This defeats the purpose of finetuning
model.config.use_cache = False

# train the model based on the above config
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.513800
20,3.298400
30,3.357900
40,3.379100
50,3.200900
60,3.467700
70,3.064700
80,3.093200
90,2.716200
100,2.611900


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=1250, training_loss=2.7754057983398437, metrics={'train_runtime': 3793.4306, 'train_samples_per_second': 1.318, 'train_steps_per_second': 0.33, 'total_flos': 1.199753527296e+16, 'train_loss': 2.7754057983398437, 'epoch': 1.0})

In [21]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import BitsAndBytesConfig,LlamaTokenizer
# from peft import PeftModel

# base_model_id = "meta-llama/Llama-2-7b-chat-hf"

# nf4Config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False,
#                                            trust_remote_code=True,
#                                            add_eos_token=True)

# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_id,  #same as before
#     quantization_config=nf4Config,  #same quantization config as before
#     device_map="auto",
#     trust_remote_code=True,
#     use_auth_token=True
# )

# tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False,
#                                            trust_remote_code=True)

# Change model checkpoint that has least training loss in the code below
# beware of overfitting!
# modelFinetuned = PeftModel.from_pretrained(model,"finetunedModel/checkpoint-1455")

In [22]:
modelFinetuned = PeftModel.from_pretrained(
    model,
    "finetunedModel/checkpoint-1250"  # Use final checkpoint instead of hardcoded number
)

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', '

In [30]:
### ENTER YOUR QUESTION BELOW

question = "Just answer this question: Tell me about the role of Maui Emergency Management Agency (MEMA) in the 2023 wildfires??"

# Format the question
eval_prompt = f"{question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024, temperature=0.7, top_p=0.9, repetition_penalty=1.1)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Just answer this question: Tell me about the role of Maui Emergency Management Agency (MEMA) in the 2023 wildfires??


Alright, so I need to figure out how to answer the user's query. They initially asked about MEMA's role during 2023 wildfires, but now they're asking again. It seems like a follow-up or maybe they want more detailed information.

I should consider what changes have happened since the first question. For instance, the 2023 wildfires were due to climate change, which might be different from previous years when natural disasters were more frequent and caused by human activities. So, I'll need to update my response to reflect that context.

Also, there was mention of the COVID-19 pandemic affecting MEMA operations before 2023. Now, focusing on 2023, it's important to highlight how climate change has led to new challenges for the agency. Maybe discussing specific incidents, like Mount Tamalpais, would add depth.

The user might be looking for updates on how MEMA handles eme

In [26]:
### RUN THIS CELL AND ENTER YOUR QUESTION IN THE POP-UP BOX

# User enters question below
user_question = input("Enter your question: ")

# Format the question
eval_prompt = f"Just answer this question concisely: {user_question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024, temperature=0.7, top_p=0.9, repetition_penalty=1.1)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Enter your question: What are the prospects for the securities industry in 2018 based on the views of industry analysts and the text?


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Just answer this question concisely: What are the prospects for the securities industry in 2018 based on the views of industry analysts and the text?


Okay, I need to figure out how to approach this user's query. Let me read through their initial question again.

The user asked, "What are the prospects for the securities industry in 2018?" They mentioned wanting a concise answer based on analyst opinions and provided text. 

I notice that they used the word "prospects" which might imply a positive outlook but could also be neutral. Maybe they're looking for a balanced view considering both potential growth and challenges.

Now, looking at my response earlier, it was neutral, just stating they expect growth without specific details. That aligns with what the user wants because they want a general overview from analysts.

But wait, perhaps the user is expecting more detailed insights. So why did I choose a neutral stance? It's probably to avoid overstepping based on limited information 

In [25]:
# User enters question below
user_question = "Summarize the officer accounts of the wildfire in Hawaii?"

# Format the question
eval_prompt = f"Just answer this question accurately: {user_question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024, temperature=0.7, top_p=0.9, repetition_penalty=1.1)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Just answer this question accurately: Summarize the officer accounts of the wildfire in Hawaii?


Alright, let me try to figure out what happened. The user asked about summarizing the officer accounts of a wildfire in Hawaii. So they're probably looking for details on who was involved and what happened after the fire.

First, I need to recall the key points from the latest reports or news about that event. I know that the wildfire was related to the 2019 Pandemic, so it's likely connected to COVID-19 spread. That makes sense because many fires are linked to environmental factors like pollution, lack of resources, etc.

I should check if there were any major incidents involving officers. Maybe there were multiple incidents where officers responded to the fire. Also, considering the size of the area, which is around 36 square miles, the response times might have been relatively quick.

Another thing to consider is how the fire spread. Was it due to wind, smoke, or something else? Also, h

In [27]:
from huggingface_hub import login
login()  # Get token from huggingface.co/settings/tokens

In [28]:
modelFinetuned.save_pretrained("DeepSeek-R1-Distill-Qwen-1.5B-finance-v1")
tokenizer.save_pretrained("DeepSeek-R1-Distill-Qwen-1.5B-finance-v1")

modelFinetuned.push_to_hub("abhi9ab/DeepSeek-R1-Distill-Qwen-1.5B-finance-v1")
tokenizer.push_to_hub("abhi9ab/DeepSeek-R1-Distill-Qwen-1.5B-finance-v1")

adapter_model.safetensors:   0%|          | 0.00/73.9M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abhi9ab/DeepSeek-R1-Distill-Qwen-1.5B-finance-v1/commit/cd2ad8a2df52254084fd5e4f4fd28d80dc050cd1', commit_message='Upload tokenizer', commit_description='', oid='cd2ad8a2df52254084fd5e4f4fd28d80dc050cd1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abhi9ab/DeepSeek-R1-Distill-Qwen-1.5B-finance-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='abhi9ab/DeepSeek-R1-Distill-Qwen-1.5B-finance-v1'), pr_revision=None, pr_num=None)

In [29]:
torch.cuda.empty_cache()